# BERT for fake news detection


Make sure that you are using GPU

In [49]:
import torch 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

You can see the type of gpu available for you:

In [50]:
!nvidia-smi

Thu Mar 24 08:52:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    72W / 149W |   2831MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 1. Load the data

In [51]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')
root='drive/My Drive/Colab Notebooks/'
paht = 'drive/My Drive/Colab Notebooks/fakkedit/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
MULTIMODAL_ONLY = True # if False, we will load all the
nameFile = 'multimodal_'
if not MULTIMODAL_ONLY:
    nameFile ='all_'


In [53]:
import pandas as pd
train = pd.read_csv(root+"data/fakeddit/" + nameFile+ "train.tsv", sep='\t')
val  = pd.read_csv(root+"data/fakeddit/" + nameFile+ "validate.tsv", sep='\t')
test  = pd.read_csv(root+"data/fakeddit/" + nameFile+ "test_public.tsv", sep='\t')

print("Dataset: ", nameFile)
print('size training: ', len(train))
print('size validation: ', len(val))
print('size test: ', len(test))


Dataset:  multimodal_
size training:  564000
size validation:  59342
size test:  59319


To make faster experimentation, we reduce the size of the datasets (of course, the results will be worse)

In [54]:
import numpy as np
reduction = 10 #100, 10 or 1

print("reduction applied: ", reduction)


print(f"Training patterns before reduction: {len(train)}")
train = train.sample(int(len(train)/reduction), random_state=12345)
print(f"Training patterns after reduction:  {len(train)}")

print(f"Validation patterns before reduction: {len(val)}")
val = val.sample(int(len(val)/reduction), random_state=12345)
print(f"Validation patterns after reduction:  {len(val)}")

print(f"Test patterns before reduction: {len(test)}")
test = test.sample(int(len(test)/reduction), random_state=12345)
print(f"Test patterns after reduction:  {len(test)}")

Training patterns before reduction: 564000
Training patterns after reduction:  56400
Training patterns before reduction: 59342
Training patterns after reduction:  5934
reduction applied:  10
Test patterns before reduction: 59319
Test patterns after reduction:  5931


In [55]:
train.head()

,author,clean_title,created_utc,domain,hasImage,id,image_url,linked_submission_id,num_comments,score,subreddit,title,upvote_ratio,2_way_label,3_way_label,6_way_label
224716,T3nd0o,this basket made out of old candy wrappers fea...,1.566649e+09,i.redd.it,True,cusqgc,https://preview.redd.it/vz64u4m03ei31.jpg?widt...,NaN,2.0,20,mildlyinteresting,This basket made out of old candy wrappers (fe...,0.80,1,0,0
525216,TindalosKeeper,ugh shitty connection,1.560360e+09,NaN,True,eqwq1xm,https://i.imgur.com/jhHIGoj.jpg,bzptls,NaN,8,psbattle_artwork,"Ugh, shitty connection!",NaN,0,2,4
104777,ZAPP3Rx,evil racoon,1.508945e+09,i.redd.it,True,78od95,https://preview.redd.it/as247oqduztz.jpg?width...,NaN,0.0,4,pareidolia,Evil racoon,0.70,0,2,2
467061,Count_Mirchaud,german tank ace michael wittmans last stand ag...,1.481311e+09,imgur.com,True,5hfnof,https://external-preview.redd.it/F4ThHaQXXlqf3...,NaN,0.0,9,fakehistoryporn,German Tank Ace Michael Wittman's last stand a...,0.92,0,2,2
350361,GuysnDolls,the dread of lancaster bay,1.552235e+09,NaN,True,ei7pl4z,https://i.imgur.com/RgHJ9D3.jpg,azb1dq,NaN,11,psbattle_artwork,The Dread of Lancaster Bay,NaN,0,2,4


We only keep the clean title and the columns for classification:

In [56]:
train_data = train[["clean_title", "2_way_label", "3_way_label", "6_way_label"]]
val_data=val[["clean_title", "2_way_label", "3_way_label", "6_way_label"]]
test_data=test[["clean_title", "2_way_label", "3_way_label", "6_way_label"]]
# remove records with Nan values
train_data = train_data.dropna()
val_data = val_data.dropna()
test_data = test_data.dropna()


In [57]:
train_data.head()

,clean_title,2_way_label,3_way_label,6_way_label
224716,this basket made out of old candy wrappers fea...,1,0,0
525216,ugh shitty connection,0,2,4
104777,evil racoon,0,2,2
467061,german tank ace michael wittmans last stand ag...,0,2,2
350361,the dread of lancaster bay,0,2,4


We save the texts:

In [58]:
X_train = train_data["clean_title"].values.tolist()
X_val = val_data["clean_title"].values.tolist()
X_test = test_data["clean_title"].values.tolist()
print(type(X_train))

<class 'list'>


We show some examples:

In [59]:
print(X_train[0])
print(X_val[0])
print(X_test[0])


this basket made out of old candy wrappers feat some hot garlic bread
catana contender for mortal kombat
cutout


We decide what type of classication we will address: 
- binary (2_way_label)
- tres classes (3_way_label)
- 6 classes (6_way_label)

In [60]:
TYPE_CLASSIFICATION = 6 # 2, 3 OR 6

In [61]:
y_train = train_data[str(TYPE_CLASSIFICATION) + '_way_label'].values.tolist()
y_val = val_data[str(TYPE_CLASSIFICATION) + '_way_label'].values.tolist()
y_test = test_data[str(TYPE_CLASSIFICATION) + '_way_label'].values.tolist()


We do not need to encoder the labels because they are already provided as numbers. 

That is, in the 6_way_label, the labels are 0, 1, 2, 3, 4, 5. 
- 0: true
- 1: Satire/Parody:
- 2: Misleading Content
- 3: Imposter Content
- 4: False Connection
- 5: Manipulated Content

In the 3_way_label, the labels are 0, 1, 2:
- 0: true
- 1: the sample is fake and contains text that is true (i.e. direct quotes from propaganda posters)
- 2: False

In the 2_way_label, the labels are 0, 1:
- 0: true
- 1: False


In [62]:
labels_dict = {}
if TYPE_CLASSIFICATION == 2:
    labels_dict[0] = "True"
    labels_dict[1] = "False"
elif TYPE_CLASSIFICATION == 3:
    labels_dict[0] = "True"
    labels_dict[1] = "Fake contains True"
    labels_dict[2] = "False"
elif TYPE_CLASSIFICATION == 6:
    labels_dict[0] = "True"
    labels_dict[1] = "Satire/Parody"
    labels_dict[2] = "Misleading Content"
    labels_dict[3] = "Imposter Content"
    labels_dict[4] = "False Connection"
    labels_dict[5] = "Manipulated Content"

print(labels_dict)

{0: 'True', 1: 'Satire/Parody', 2: 'Misleading Content', 3: 'Imposter Content', 4: 'False Connection', 5: 'Manipulated Content'}


We show some examples and their labels:

In [63]:
print(X_train[0], y_train[0], labels_dict[y_train[0]])
print(X_val[0], y_val[0], labels_dict[y_val[0]])
print(X_test[0], y_test[0], labels_dict[y_test[0]])




this basket made out of old candy wrappers feat some hot garlic bread 0 True
catana contender for mortal kombat 4 False Connection
cutout 4 False Connection


## Data Visualization

Here we should provide more detail about the dataset (class distribution, average lenght of the texts, etc)

In [64]:
# we will do it after

## Transformers

First, we install the libraries:

In [65]:
!pip3 install transformers numpy torch sklearn


To make our experiments reproducible, we set a random seed:


In [66]:
import torch
import numpy as np
torch.manual_seed(0)
np.random.seed(0)

Import the necessary modules:

In [67]:
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments



we'll be using the BERT model. More specifically, we'll be using bert-base-uncased pre-trained weights from the library.

In [68]:
# the model we gonna train, base uncased BERT
# check text classification models here: https://huggingface.co/models?filter=text-classification
model_name = "bert-base-uncased"
# max sequence length for each document/sentence sample
max_length = 512

Download the tokenizer provided by this model:

In [69]:
# load the tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a

Now let's use our tokenizer to encode our corpus. This tokenizer translate the tokens to ids. We set truncation to True so that we eliminate tokens that go above max_length, we also set padding to True to pad documents that are less than max_length with empty tokens.

In [70]:
# tokenize the dataset, truncate when passed `max_length`, 
# and pad with 0's when less than `max_length`
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=max_length)
val_encodings = tokenizer(X_val, truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=max_length)


The below code wraps our tokenized text data into a torch Dataset. Since we gonna use Trainer from Transformers library, it expects our dataset as a torch.utils.data.Dataset, so we made a simple class that implements the len() method that returns the number of samples, and getitem() method to return a data sample at a specific index.

In [71]:
class TorchDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_torch_dataset = TorchDataset(train_encodings, y_train)
val_torch_dataset = TorchDataset(val_encodings, y_val)
test_torch_dataset = TorchDataset(test_encodings, y_test)

In [72]:
print(len(train_encodings.encodings), len(y_train))

56400 56400


## Training the model

Now that we have our data prepared, let's download and load our BERT model and its pre-trained weights. We're using BertForSequenceClassification class from Transformers library, we set num_labels to the length of our available labels, in this case, 2.

We also cast our model to our CUDA GPU. If you're on CPU (not suggested), then just delete to() method.

In [73]:
# load the model and pass to CUDA
model = BertForSequenceClassification.from_pretrained(model_name, num_labels = TYPE_CLASSIFICATION).to("cuda")

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "p

Before we start fine-tuning our model, let's make a simple function to compute the metrics we want. In this case, accuracy.
You're free to include any metric you want, I've included accuracy, but you can add precision, recall, etc.

In [74]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [75]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,   # 3           # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=400,               # log & save weights each logging_steps
    save_steps=400,
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

using `logging_steps` to initialize `eval_steps` to 400
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [76]:
trainer = Trainer(
    model = model,                         # the instantiated Transformers model to be trained
    args = training_args,                  # training arguments, defined above
    train_dataset = train_torch_dataset,         # training dataset
    eval_dataset = val_torch_dataset,          # evaluation dataset
    compute_metrics = compute_metrics,     # the callback that computes metrics of interest
)

# train the model
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 56400
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7050


Step,Training Loss,Validation Loss,Accuracy
400,1.287100,1.007524,0.651163
800,0.976500,0.971207,0.682171
1200,0.881300,0.845004,0.699528
1600,0.849300,0.786233,0.723964
2000,0.768300,0.788771,0.726323
2400,0.765000,0.736870,0.742669
2800,0.777400,0.751365,0.740647
3200,0.737400,0.786757,0.728008
3600,0.761700,0.707152,0.748905
4000,0.730800,0.686824,0.756825


***** Running Evaluation *****
  Num examples = 5934
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-400
Configuration saved in ./results/checkpoint-400/config.json
Model weights saved in ./results/checkpoint-400/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5934
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-800
Configuration saved in ./results/checkpoint-800/config.json
Model weights saved in ./results/checkpoint-800/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5934
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-1200
Configuration saved in ./results/checkpoint-1200/config.json
Model weights saved in ./results/checkpoint-1200/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5934
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-1600
Configuration saved in ./results/checkpoint-1600/config.json
Model weights saved in ./results/checkpoint-1600/pytorch_model.bi

TrainOutput(global_step=7050, training_loss=0.7742142892391124, metrics={'train_runtime': 5185.1111, 'train_samples_per_second': 10.877, 'train_steps_per_second': 1.36, 'total_flos': 5738904885859200.0, 'train_loss': 0.7742142892391124, 'epoch': 1.0})

In [77]:
# evaluate the current model after training
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 5934
  Batch size = 20


{'epoch': 1.0,
 'eval_accuracy': 0.7736771149309066,
 'eval_loss': 0.63472980260849,
 'eval_runtime': 46.9004,
 'eval_samples_per_second': 126.523,
 'eval_steps_per_second': 6.333}

Save the model

In [78]:
# saving the fine tuned model & tokenizer
model_path = "fake-bert-base-uncased"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Configuration saved in fake-bert-base-uncased/config.json
Model weights saved in fake-bert-base-uncased/pytorch_model.bin
tokenizer config file saved in fake-bert-base-uncased/tokenizer_config.json
Special tokens file saved in fake-bert-base-uncased/special_tokens_map.json


('fake-bert-base-uncased/tokenizer_config.json',
 'fake-bert-base-uncased/special_tokens_map.json',
 'fake-bert-base-uncased/vocab.txt',
 'fake-bert-base-uncased/added_tokens.json',
 'fake-bert-base-uncased/tokenizer.json')

In [79]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    # return probs.argmax() is a tensor. We have to return its item
    return probs.argmax().item()


In [80]:
y_pred=[]
for text in X_test:
    y_pred.append(get_prediction(text))


In [81]:
from sklearn.metrics import confusion_matrix, classification_report
print(labels_dict)
print(classification_report(y_test, y_pred))


{0: 'True', 1: 'Satire/Parody', 2: 'Misleading Content', 3: 'Imposter Content', 4: 'False Connection', 5: 'Manipulated Content'}
              precision    recall  f1-score   support

           0       0.80      0.87      0.83      2351
           1       0.79      0.46      0.58       376
           2       0.71      0.60      0.65      1154
           3       0.57      0.23      0.33       139
           4       0.77      0.87      0.82      1675
           5       0.78      0.72      0.75       236

    accuracy                           0.77      5931
   macro avg       0.73      0.62      0.66      5931
weighted avg       0.77      0.77      0.76      5931



We show some predictions for new texts. The second prediction is wrong (the news is fake). 

In [82]:
text = "Donald Trump sent his own plane to transport 200 stranded marines."

pred=get_prediction(text)
print('class:', labels_dict[pred])

text = "FBI director received millions from Clinton Foundation, his brother’s law firm does Clinton’s taxes"

pred=get_prediction(text)
print('class:', labels_dict[pred])

class: True
class: True
